In [1]:
from html import unescape

In [61]:
from tempfile import TemporaryDirectory
from xml.etree import cElementTree
from itertools import chain
from pathlib import Path
from io import StringIO
import subprocess
import shutil
import os

from typing import Optional, Generator, Tuple


def ned_to_xml(ned_file: str | Path) -> cElementTree.Element:
    if not isinstance(ned_file, Path):
        ned_file = Path(ned_file)

    opp_nedtool_path = Path(os.environ['HOME']) \
        .joinpath('bin/opp_nedtool.exe') \
        .as_posix()

    with TemporaryDirectory() as temp_dir:
        temp_ned_file = Path(temp_dir).joinpath(ned_file.name)

        temp_xml_file = temp_ned_file.with_suffix(
            f'{temp_ned_file.suffix}.xml'
        )

        shutil.copy(ned_file, temp_ned_file)

        subprocess.Popen([
            opp_nedtool_path,
            'convert',
            temp_ned_file
        ]).communicate()

        xml = cElementTree.parse(temp_xml_file).getroot()
        xml.attrib['filename'] = ned_file.as_posix()

        return xml

def find_ned_xml_entities(xml: cElementTree.Element) -> Generator[cElementTree.Element, None, None]:
     yield from chain(
        xml.iter(tag='simple-module'),
        xml.iter(tag='compound-module'),
        xml.iter(tag='channel'),
    )

def unescape_xml(xml: cElementTree.Element) -> None:
    for element in xml.iter():
        if element.text is not None:
            element.text = unescape(element.text)
    
        for key in ['content', 'value']:
            if key in element.attrib:
                element.set(
                    key,
                    unescape(element.get(key))
                )

def find_ned_xml_import_files(
    xml: cElementTree.Element,
    package_root: str | Path,
    package_name: str
) -> Generator[Tuple[str, cElementTree.Element], None, None]:
    if not isinstance(package_root, Path):
        package_root = Path(package_root)

    for import_element in xml.iter(tag='import'):
        import_path = import_element \
            .get('import-spec') \
            .split('.')
        
        if import_path[0] == package_name:
            import_path = import_path[1:]
        
        *import_pieces, ned_name = import_path
        
        if ned_name == '**':
            glob_string = '/'.join([*import_pieces, '**/*.ned'])
        else:
            glob_string = '/'.join([*import_pieces, '*.ned'])
        
        if ned_name in ['*', '**']:
            for sub_ned_file in package_root.glob(glob_string):
                yield sub_ned_file
        else:
            for sub_ned_file in package_root.glob(glob_string):
                sub_ned_file_xml = ned_to_xml(sub_ned_file)
    
                for element in find_ned_xml_entities(sub_ned_file_xml):
                    if ned_name == element.get('name'):
                        yield sub_ned_file, import_element

def get_xml_element_parent(
    xml: cElementTree.Element,
    element: cElementTree.Element
) -> cElementTree.Element:
    for xml_element in xml.iter():
        for i, child_xml_element in enumerate(xml_element):
            if child_xml_element == element:
                return i, xml_element

    raise ValueError('The XML element is not in the provided tree')

In [62]:
def recursively_load_ned_to_xml(
    ned_file: str | Path,
    *,
    package_root: Optional[Path]=None,
    package_name: Optional[str]=None
) -> cElementTree.Element:
    if not isinstance(ned_file, Path):
        ned_file = Path(ned_file)

    xml = ned_to_xml(ned_file)
    unescape_xml(xml)

    if package_name is None:
        is_first_call = True

        package_name = xml \
            .find('.//package') \
            .get('name')

        for ned_xml_entity in find_ned_xml_entities(xml):
            ned_xml_entity.set('package', package_name)
    else:
        is_first_call = False

    if package_root is None:
        package_root = Path(ned_file).parent

    new_element_offset = 0

    for sub_ned_file, element in find_ned_xml_import_files(xml, package_root, package_name):
        sub_ned_file_xml = recursively_load_ned_to_xml(
            sub_ned_file,
            package_root=package_root,
            package_name=package_name
        )

        sub_ned_file_package_root = sub_ned_file_xml \
            .find('.//package') \
            .get('name')

        index, parent = get_xml_element_parent(xml, element)
        index += new_element_offset

        parent.remove(element)
        new_element_offset -= 1

        for imported_element in find_ned_xml_entities(sub_ned_file_xml):
            if not any(
                (
                    m.tag == imported_element.tag and
                    m.get('name') == imported_element.get('name')
                ) for m in find_ned_xml_entities(parent)
            ):
                if 'package' not in imported_element.attrib:
                    imported_element.set(
                        'package',
                        sub_ned_file_package_root
                    )

                parent.insert(index, imported_element)

                new_element_offset += 1
                index += 1

    if is_first_call:
        for network in xml.iterfind(
            './/compound-module/parameters/property[@name="isNetwork"]....'
        ):
            parameters = network.find('./parameters')
            property = parameters.find(
                './property[@name="isNetwork"]'
            )
    
            parameters.remove(property)
            network.tag = 'network'
    
        for package in xml.iterfind('./package'):
            xml.remove(package)

        for parent_element in xml.iterfind('.//*/comment/..'):
            for comment_element in parent_element.iterfind('./comment'):
                parent_element.remove(comment_element)

        for i, element in enumerate(
            sorted(xml.iterfind('./*'),
                   key=lambda t: t.get('package', '_'))
        ):
            xml[i] = element
    
        cElementTree.indent(xml)

    return xml

In [63]:
ned_file = 'C:/Users/riley/Projects/covid19_guidance_simulator/src/package.ned'

In [64]:
xml = recursively_load_ned_to_xml(ned_file)

In [56]:
print(cElementTree.tostring(xml).decode())

<ned-file filename="C:/Users/riley/Projects/covid19_guidance_simulator/src/package.ned">
  <network name="Covid19GuidanceSimulator" package="covid19_guidance_simulator">
    <parameters>
      <param type="string" name="start_date" value="&quot;2020-01-01&quot;" is-default="true" />
      <param type="string" name="map_color_mode" value="&quot;dark&quot;" is-default="true" />
      <param type="string" name="short_name" value="&quot;No Name&quot;" is-default="true" />
      <param type="string" name="long_name" value="&quot;No Name&quot;" is-default="true" />
      <param type="string" name="map_name" value="&quot;new_york_city&quot;" is-default="true" />
      <param type="int" name="num_counties" value="intuniform(2, 5)" is-default="true">
                </param>
      <property name="display">
        <property-key>
          <literal type="string" text="&quot;bgb=915,550;bgi=images/maps/$map_color_mode/$map_name,s;bgg=100,2,grey95;bgu=km&quot;" value="bgb=915,550;bgi=images/maps/$